# Monty Hall

Below are some simulations for one of the lab exercises. Sometimes, we can't easily find an answer to a probability  problem using theory. This notebook shows how we can find some answers using simulation instead.

First, we need to import some libraries.

In [1]:
import numpy as np  # Math
import random       # Randomness
import matplotlib.pylab as plt  # Plotting
import ipywidgets as widgets    # Interactive stuff

A good first step is to define the set of outcomes. We can define these as tuples as ("chosen door", "opened door", "switched door"), printed by the code below.

In [2]:
outcomes = list()
for door in [1, 2, 3]:
  for choice in [1, 2, 3]:
    for change in [True, False]:
      outcomes.append((door, choice, change))

print("%i outcomes: %s " % (len(outcomes), outcomes))

18 outcomes: [(1, 1, True), (1, 1, False), (1, 2, True), (1, 2, False), (1, 3, True), (1, 3, False), (2, 1, True), (2, 1, False), (2, 2, True), (2, 2, False), (2, 3, True), (2, 3, False), (3, 1, True), (3, 1, False), (3, 2, True), (3, 2, False), (3, 3, True), (3, 3, False)] 


Next, we need to define a win contition. I've chosen to do this as a python function with `if` statements returning a boolean (i.e. True/False). It is good practice to always try out your code in a controlled setting before using it as a part of something larger. Especially when it is so easy to do. To be clear **do not trust your code, always test it**.

In [3]:
def did_we_win(outcome):
  door, choice, change = outcome
  if door==choice and not change:
    return True
  elif door != choice and change:
    return True
  else:
    return False

for outcome in random.choices(outcomes, k=5):
  print("Outcome drawn: %s" % repr(outcome))
  door, choice, change = outcome
  print(" Door with car:", door)
  print(" We chose door:", choice)
  if change:
    print(" We changed our choice")
  else:
    print(" We stood by our choice")
  if did_we_win(outcome):
    print("  -> We won a car!")
  else:
    print("  -> We won a goat :(")
  print()

Outcome drawn: (1, 2, False)
 Door with car: 1
 We chose door: 2
 We stood by our choice
  -> We won a goat :(

Outcome drawn: (3, 1, False)
 Door with car: 3
 We chose door: 1
 We stood by our choice
  -> We won a goat :(

Outcome drawn: (1, 2, False)
 Door with car: 1
 We chose door: 2
 We stood by our choice
  -> We won a goat :(

Outcome drawn: (3, 3, True)
 Door with car: 3
 We chose door: 3
 We changed our choice
  -> We won a goat :(

Outcome drawn: (1, 2, True)
 Door with car: 1
 We chose door: 2
 We changed our choice
  -> We won a car!



### A fast way of finding the probability.

We can assume all outcomes are equally likely and simply go through them. This is a big assumption, why can we use it here? Is the assumption always valid?
 
**Relevant theory:**
 
Conditional probability is defined as: $P(A|B) = \frac{P(A \cap B)}{P(B)}$. If we define winning as event A and changing door as event B, event B's compliment will be not changing door. Since each outcome is either in B or not, then $P(B^c)=1-P(B)$.

In [4]:
A_and_B = 0
A_and_Bc = 0
B = 0
for outcome in outcomes:
  door, choice, change = outcome
  if did_we_win(outcome): # Event A, i.e. winning
    if change:            # Event A and B
      A_and_B += 1
    else:                 # Event A and B^c, i.e. not change door
      A_and_Bc += 1
  if change:              # Event B
    B += 1

print("p(win | changed door)    = %.1f%%" % (100*A_and_B/B))
print("p(win | not change door) = %.1f%%" % (100*A_and_Bc/(len(outcomes)-B)))

p(win | changed door)    = 66.7%
p(win | not change door) = 33.3%


### A fun way of finding the probability 

We can find the probabilities by simulating reality. By doing this, we should *converge* to the theoretical number above with time.

The following code will let you play a set number of game rounds and see how the probabilities change.

In [5]:
data = list()

def run_game_show(n_rounds):
  # Run the show
  new_data = random.choices(outcomes, k=n_rounds)
  data.extend(new_data)
  print("New data (n=%i): %s" % (len(new_data), new_data))
  if len(data) < 50:
    print("All datapoints:", data)
  else:
    print("Last data points: ...", data[-10:])
  # Define the figures
  fig = plt.figure(figsize=(15, 5), dpi=80)
  ax = fig.subplots(1, 2)
  ax[0].set_ylabel("# shows")
  losses = np.sum([not did_we_win(outcome) for outcome in data])
  ax[0].set_xlabel("Outcomes (%i losses not counted)" % losses)
  ax[1].set_xlabel("# shows")
  ax[1].set_ylabel('Probability')
  # Plot data

  strategy_counts = np.zeros(2)
  new_strategy_counts = np.zeros(2)
  games = np.arange(1, len(data)+1)
  labels = ['win | change', 'win | no change']
  colours = ["C0", "C1"]

  strategy_counts[0] = np.sum([did_we_win(outcome) and outcome[2] for outcome in data])
  strategy_counts[1] = np.sum([did_we_win(outcome) and not outcome[2] for outcome in data])
  new_strategy_counts[0] = np.sum([did_we_win(outcome) and outcome[2] for outcome in new_data])
  new_strategy_counts[1] = np.sum([did_we_win(outcome) and not outcome[2] for outcome in new_data])
  old_strategy_counts = strategy_counts-new_strategy_counts
  ax[0].bar(labels, new_strategy_counts, bottom=old_strategy_counts, align='center', color=colours, alpha=.5)
#  ax[0].bar(labels, new_strategy_counts, bottom=old_strategy_counts, align='center', color='red', alpha=.6)
  ax[0].bar(labels, old_strategy_counts, align='center', color=colours, alpha=.8)
  
  A_B = np.cumsum([did_we_win(outcome) and outcome[2] for outcome in data])
  A_Bc = np.cumsum([did_we_win(outcome) and not outcome[2] for outcome in data])
  B = np.cumsum([outcome[2] for outcome in data])
  Bc = np.cumsum([not outcome[2] for outcome in data])
  ax[1].plot(games, A_B/B, alpha=.7, label="P(win|change)")
  ax[1].plot(games, A_Bc/Bc, alpha=.7, label="P(win|no change)")
  ax[1].plot(games, 2/3*np.ones(games.shape), 'g--', alpha=.8, label="Theoretical P(win|change)")
  ax[1].plot(games, 1/3*np.ones(games.shape), 'r--', alpha=.8, label="Theoretical P(win|no change)")
  # Adjust graphs and commit to screen
  a = list(ax[0].axis())
  a[2] = 0
  a[3] = max(np.max(strategy_counts)+10, 25)
  ax[0].axis(tuple(a))
  a = list(ax[1].axis())
  a[2] = 0
  a[3] = 1
  ax[1].axis(tuple(a))
  ax[1].legend(loc='upper right')
  fig.show()

widgets.interact_manual.opts['manual_name'] = 'Play rounds!'
interact_plot = widgets.interact_manual(run_game_show, n_rounds=widgets.IntSlider(min=1, max=100, step=1, value=1));
output = interact_plot.widget.children[-1] # This should prevent flickering
output.layout.height = '400px'

interactive(children=(IntSlider(value=1, description='n_rounds', min=1), Button(description='Play rounds!', st…

Can you see the effect of the *law of large numbers*?